<a href="https://colab.research.google.com/github/DanielSchussmann/Seminar/blob/main/Seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $$\text{Machine Learning updates} $$

##$$\text{ Variables}$$ 

$$ i,k = \text{parameter for the layer}$$
$$ j,l= \text{parameter for the position inside the i-th layer}$$
$$ b_{i,j} = \text{Bias of neuron} $$

$$ x_{i,j} = \text{input value / fireing rate}$$
$$ w_{i,j,k,l} = \text{weight of connection, k and l are destination neuron}$$
$$\sigma = \text{activation function}$$
$$n_i = \text{number of neurons in the i-th layer}$$

(add illustration for clarification)

$$x_{i+1,j} = \sigma [\sum_{j=1}^{n_i}(w_{(i,j,k,l)}^T * x_{i,j}) +b_{i+1,j}] \text{(not a recursive expression)}$$
$$ .\\.\\.\\ x_{1,1} = \sigma [\sum_{j=1}^{n_0}(w_{(0,j,1,1)}^T * x_{0,j}) +b_{1,1}] \\ $$
$$ x_{2,4} = \sigma [\sum_{j=1}^{n_1}(w_{(1,j,2,4)}^T * x_{1,j}) +b_{2,4}] \\ $$

In [ ]:
import numpy as np

a= np.random.randn(2,4) #creates a Matrix of Standard distributed values maching the paramter dimensions
a

array([[-0.87302466,  1.17997524, -0.33669142, -0.77630768],
       [ 1.16224348, -0.28885221,  0.43584297,  1.20120941]])